**WARNING: Before you run the project, you should run nltk.download("all") code in order to download nltk package. Training usually takes between 10-12 minutes.**

In [ ]:
import nltk
nltk.download("all")

In [ ]:
import json
import os
import nltk
import random
from nltk.util import ngrams
from nltk.lm import Laplace
from nltk.lm.preprocessing import padded_everygram_pipeline

# Reading json files from file. This file was in my google drive. Make sure file name is correct and json files are in there.
dir = "/content/drive/MyDrive/2021-01-json-9k"
jsons = []
for filename in os.listdir(dir):
  if filename.endswith(".json"):
    jsonpath = os.path.join(dir, filename)
    with open(jsonpath, "r") as jsonfile:
            myjsonfile = json.load(jsonfile)
            jsons.append(myjsonfile)
tokens_no_stop_words = []
stop_words = nltk.corpus.stopwords.words('turkish')
chunk = [".",",","/","[","]",":",";","..."]

#Reading words from json files and tokenizing them. Then removing expressions like "." , " /" etc and stop words.
for json in jsons:
  temptext = ""
  for i in json["vurgular"]:
    temptext += i + " "
  temptext += json["Mahkemesi"] + " " + json["Dairesi"] + " " + json["Esas"] + " " + json["Birinci Mahkemesi"] + " " + json["Karar"] + " " + json["Mahkeme Günü"] + " " + json["Mahkeme Ayı"] + " " + json["Mahkeme Yılı"] + " " + json["Suç"] + " " + json["Hüküm"] + " " + json["Dava Türü"] + " " + json["ictihat"] + " "
  temptext = temptext.lower()
  tokens = nltk.word_tokenize(temptext)
  for i in tokens:
      if i in chunk:
        tokens.remove(i)
  for i in tokens:
     if i not in stop_words:
        tokens_no_stop_words.append(i)

# Splitting words into test and training sets for word based n-gram model.
training_size_for_word_based = int(len(tokens_no_stop_words) * 0.8)
training_set_for_word_based = tokens_no_stop_words[:training_size_for_word_based]
test_set_for_word_based = tokens_no_stop_words[training_size_for_word_based:]

#Forming 2-grams and training model.
training_ngrams_for_word_based, vocab = padded_everygram_pipeline(2, training_set_for_word_based)
word_based_model = Laplace(1)
word_based_model.fit(training_ngrams_for_word_based,vocab)

#Testing word based n-gram model
test_ngrams_for_word_based = ngrams(test_set_for_word_based,2)
perplexity = word_based_model.perplexity(test_ngrams_for_word_based)
print("Perplexity for word based model: ", perplexity)

# Tokenizing words into characters.
characters_from_tokenized_no_stop_words = []
for i in tokens_no_stop_words:
  for j in i:
    characters_from_tokenized_no_stop_words.extend(j)

#Splitting characters into training and test sets.
training_size_for_character_based = int(len(characters_from_tokenized_no_stop_words) * 0.8)
training_set_for_character_based = characters_from_tokenized_no_stop_words[:training_size_for_character_based]
test_set_for_character_based = characters_from_tokenized_no_stop_words[training_size_for_character_based:]

# Forming 2-grams for character based model and training it.
training_ngrams_for_character_based, vocab2 = padded_everygram_pipeline(2,training_set_for_character_based)
character_based_model = Laplace(1)
character_based_model.fit(training_ngrams_for_character_based,vocab2)

# Testing character based model.
test_ngrams_for_character_based = ngrams(test_set_for_character_based,2)
perplexity = word_based_model.perplexity(test_ngrams_for_character_based)
print("Perplexity for character based model: ", perplexity)





